In [ ]:
# Cargar librerias

import pandas as pd
pd.set_option('display.max_columns', 25) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Numero máximo de filas a mostar
import numpy as np
np.random.seed(3301)
import pandas as pd
# Para preparar los datos
from sklearn.preprocessing import LabelEncoder
# Para crear el arbol de decisión 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para evaluar el modelo
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix
# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
# Para la validación cruzada
from sklearn.model_selection import KFold 
#Librerías para la visualización
import matplotlib.pyplot as plt
# Seaborn
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline
import seaborn as sns 
from sklearn import tree
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
from nltk.stem.snowball import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix


In [ ]:
import sys
import numpy as np
from time import time
import pandas as pd
import warnings

In [ ]:
import seaborn as sns  # Plot utility
from sklearn.utils import shuffle # to shuffle the data    

In [ ]:
import re
import contractions

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
#from nltk import word_tokenize

In [ ]:
import inflect

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Cargar datos

df_original = pd.read_csv('MovieReviews.csv', sep=",",encoding='utf-8', index_col=0) 
df = df_original.copy()
print(df.shape)
df.head(5)

In [ ]:
df.shape

In [ ]:
df['sentimiento'] = df['sentimiento'].replace({'negativo':0, 'positivo':1})

In [ ]:
df.head(5)

In [ ]:
plt.figure(figsize=(6,5))
plt.title("Número de valores positivos y negativos de sentimiento")
plot = sn.countplot(x='sentimiento', data=df)
for p in plot.patches:
    plot.annotate(p.get_height(), (p.get_x() + 0.1, p.get_height() + 50))

In [ ]:
from langdetect import detect
# Create a new column 'language' in the dataframe
df['language'] = df['review_es'].apply(lambda x: detect(x))
# Print the number of reviews in each language
print(df['language'].value_counts())

In [ ]:
df.head()

In [ ]:
df = df[df['language'] == 'es'].drop(columns=['language'])

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
plt.figure(figsize=(6,5))
plt.title("Número de valores positivos y negativos de sentimiento")
plot = sn.countplot(x='sentimiento', data=df)
for p in plot.patches:
    plot.annotate(p.get_height(), (p.get_x() + 0.1, p.get_height() + 50))

## Preparación

In [ ]:
reviews_df = df.loc[df['sentimiento'].notnull()]

In [ ]:
reviews_df['sentimiento'].value_counts(dropna = False, normalize = True )

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(reviews_df['review_es'], reviews_df['sentimiento'], test_size = 0.3, stratify = reviews_df['sentimiento'], random_state = 42)
X_train.shape

In [ ]:
pd.Series(y_train).value_counts(normalize = True)

In [ ]:
X_test.shape

In [ ]:
pd.Series(y_test).value_counts(normalize = True)

## Limpieza de las palabras

### Tokenizacion

In [ ]:
def tokenize(words):
    return nltk.word_tokenize(words)

In [ ]:
X_train = X_train.apply(tokenize)
X_train.head()

### Limpieza de valores no deseados

In [ ]:
def remove_markup(words):
    words_ok = []
    for word in words:
        word = re.sub('(<.*?>)',' ', str(word))
        word = re.sub('\[[^]]*\]', ' ', word)
        words_ok.append(word)
    return words_ok
        
    
def remove_short_words(words):
    words_ok = []
    for word in words:
        if len(word) > 1: #Only words with size>1
            words_ok.append(re.sub('[^A-Za-z]+','',str(word)))
    return words_ok
def replace_numbers(words):
    p = inflect.engine()
    words_ok = []
    for word in words:
        try:
            if word.isdigit():
                words_ok.append(p.number_to_words(word))
            else:
                words_ok.append(word)
        except:
            print('[ERROR: word numOutOfRangeError...]'+str(word))#special cases with large numbers
            pass
    return words_ok

def remove_stop_words(words):
    stopWords = set(stopwords.words('spanish'))
    words_ok = []
    for w in words:
        if w not in stopWords:
            words_ok.append(w)
    return words_ok

def preprocess_words(words):
    temp = remove_short_words(words)
    temp = replace_numbers(temp)
    temp = remove_stop_words(temp)
    temp = remove_markup(temp)
    return temp

In [ ]:
X_train = X_train.apply(preprocess_words)
X_train.head()

### Normalización

In [ ]:
def lemmatize(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemmas.append(lemmatizer.lemmatize(word))
    return lemmas

In [ ]:
X_train = X_train.apply(lemmatize) 
X_train.head()

In [ ]:
X_train = X_train.apply(lambda x: ' '.join(map(str, x)))
X_train.head()

## TfIdVectorizer

In [ ]:
# Aplicar TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
print(X_test_tfidf)

## Random Forest

In [ ]:
# Mejor modelo
modeloRF = RandomForestClassifier(n_estimators=1200)
modeloRF.fit(X_train_tfidf, y_train)

y_train_pred = modeloRF.predict(X_train_tfidf)
y_test_pred = modeloRF.predict(X_test_tfidf)


In [ ]:
# Calculo de metricas
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

f1_train = f1_score(y_train, y_train_pred)
f1_test =f1_score(y_test, y_test_pred)

recall_train = recall_score(y_train, y_train_pred)
recall_test = recall_score(y_test, y_test_pred)

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_train, y_train_pred)

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred)

In [ ]:
print("Exactitud en entrenamiento: "+str(accuracy_train))
print("Exactitud en prueba: "+str(accuracy_test))

print("F1 en entrenamiento: "+str(f1_train))
print("F1 en prueba: "+str(f1_test))

print("Recall en entrenamiento: "+str(recall_train))
print("Recall en prueba: "+str(recall_test))


Almacenamiento del modelo:

In [ ]:
import joblib

joblib.dump((modeloRF,tfidf_vectorizer), "./app/random_forest.joblib")